In [1]:
from pl_model import YOLOv4PL

In [2]:
import pytorch_lightning as pl
from argparse import Namespace
from pytorch_lightning.callbacks import LearningRateLogger

In [3]:
hparams = {
    "n_classes" : 170,

    "pretrained" : False,
    "train_ds" : "train.txt",
    "valid_ds" : "valid.txt",
    "img_extensions" : [".JPG", ".jpg"],
    "bs" : 1,
    "momentum": 0.9,
    "wd": 0.001,
    "lr": 1e-8,
    "epochs" : 100,
    "pct_start" : 10/100,
    
    "optimizer" : "Ranger",
    "flat_epochs" : 50,
    "cosine_epochs" : 25,
    "scheduler" : "Cosine Delayed", 
    
    "SAT" : False,
    "epsilon" : 0.1,
    "SAM" : False,
    "ECA" : False,
    "WS" : False,
    "Dropblock" : False,
    "iou_aware" : False,
    "coord" : False,
    "hard_mish" : False,
    "asff" : False,
    "repulsion_loss" : False,
    "acff" : True,
    "bcn" : False,
    "mbn" : False,
}

In [4]:
hparams = Namespace(**hparams)
m = YOLOv4PL(hparams)

In [5]:
m.cpu();

In [6]:
tb_logger = pl.loggers.TensorBoardLogger('logs/', name = "yolov4")

In [7]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath='model_checkpoints/yolov4{epoch:02d}',
    verbose=True,
    monitor="training_loss_epoch",
    mode='min',
)

In [8]:
t = pl.Trainer(logger = tb_logger,
           checkpoint_callback=checkpoint_callback,
           gpus=0,
           precision=32,
           benchmark=True,
           callbacks=[LearningRateLogger()],
           min_epochs=100,


#            resume_from_checkpoint="model_checkpoints/yolov4epoch=82.ckpt",
        #    auto_lr_find=True,
          #  auto_scale_batch_size='binsearch',
        #    fast_dev_run=True
          )


GPU available: True, used: False
No environment variable for node rank defined. Set as 0.


In [9]:
r = t.lr_find(m, min_lr=1e-10, max_lr=1, early_stop_threshold=None)
r.plot()


    | Name                                          | Type              | Params
--------------------------------------------------------------------------------
0   | model                                         | YOLOv4            | 69 M  
1   | model.backbone                                | Backbone          | 26 M  
2   | model.backbone.d1                             | DownSampleFirst   | 61 K  
3   | model.backbone.d1.c1                          | ConvBlock         | 928   
4   | model.backbone.d1.c1.module                   | Sequential        | 928   
5   | model.backbone.d1.c1.module.0                 | Conv2d            | 864   
6   | model.backbone.d1.c1.module.1                 | BatchNorm2d       | 64    
7   | model.backbone.d1.c1.module.2                 | Mish              | 0     
8   | model.backbone.d1.c2                          | ConvBlock         | 18 K  
9   | model.backbone.d1.c2.module                   | Sequential        | 18 K  
10  | model.backbone.d1.c2.

IndexError: index 72 is out of bounds for dimension 4 with size 40

In [ ]:
t.fit(m)